# Kaggle | Enefit - Predict Energy Behavior of Prosumers
link: https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/data <BR>
**Discord**<br>
1. cudf+gpu is also a good way to preprocess faster<br>
2. Basically making tree-based method works better for TS regression, after thinking of the issue you raised yesterday

## Data dict

In [2]:
import pandas as pd
import pathlib
from zipfile import ZipFile
import calendar
import re
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import traceback
import numpy as np

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.5f}'.format)

### Functions

In [4]:
def create_data_dictionary(df: pd.DataFrame):
    try:
    # identify colames and dtypes
        data_dict = df.dtypes
        data_dict = data_dict.reset_index()
        data_dict = data_dict.rename(columns = {'index': 'Feature', 0: 'Data Type'})
        print('identify colames and dtypes: done')
    # null count
        data_dict['null_count'] = [df[(df[feature].isnull()) | (df[feature] == '')].shape[0] for feature in data_dict['Feature']]
        data_dict['null_count_perc'] = data_dict['null_count']/(df.shape[0])
        print('null count: done')
    # data length and data example
        data_length_features = [feature for feature in data_dict['Feature']]
        data_length = pd.DataFrame({'Feature': data_length_features,
                                'length_occurancy': [None]*len(data_length_features),
                                'data_example': [None]*len(data_length_features)})
        data_length['length_occurancy'] = data_length['length_occurancy'].astype('object')
        for i, k in enumerate(data_dict['Feature']):
            if np.issubdtype(df[k].dtype, np.floating):
                example = list(df[k].describe().to_dict().values())[:4]
            else:
                example = df[k].value_counts()[:4].index.tolist()
            data_length.at[i, 'data_example'] = example
            if is_numeric_dtype(df[k]):
                data_length.at[i, 'length_occurancy'] = None
            else:
                index = df[k].str.len().value_counts().index.tolist()
                values = df[k].str.len().value_counts().values.tolist()
                result = list(zip(index, values))
                data_length.at[i, 'length_occurancy'] = result
                data_dict = data_dict.merge(data_length, on= ['Feature'])
        print('data length and data example: done')
        # unique values
        data_dict['unique_values'] = [round(df[feature].nunique()/df[feature].shape[0] , 4) for feature in data_dict['Feature']]
    except Exception as e:
        print(f"An error occurred: {e}")
        traceback.print_exc()
        return None
    return data_dict

# Read file

In [5]:
root = '/home/carolinefcg/Documents/GitHub/Kaggle/predict-energy-behavior-of-prosumers/'

In [6]:
train_raw = pd.read_csv(root+'train.csv')
train_dict = create_data_dictionary(train_raw)
train_dict

identify colames and dtypes: done
null count: done
data length and data example: done


,Feature,Data Type,null_count,null_count_perc,length_occurancy,data_example,unique_values
0,county,int64,0,0.00000,None,"[0, 11, 7, 5]",0.00000
1,is_business,int64,0,0.00000,None,"[1, 0]",0.00000
2,product_type,int64,0,0.00000,None,"[3, 1, 0, 2]",0.00000
3,target,float64,528,0.00026,None,"[2017824.0, 274.8555600988986, 909.50237801986...",0.28020
4,is_consumption,int64,0,0.00000,None,"[0, 1]",0.00000
5,datetime,object,0,0.00000,"[(19, 2018352)]","[2022-11-27 12:00:00, 2022-11-17 03:00:00, 202...",0.00760
6,data_block_id,int64,0,0.00000,None,None,0.00030
7,row_id,int64,0,0.00000,None,None,1.00000
8,prediction_unit_id,int64,0,0.00000,None,None,0.00000


In [14]:
train_raw[(train_raw.is_business==1) & (train_raw.product_type==3) & (train_raw.is_consumption==1) & (train_raw.county==13)].head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
105,13,1,3,750.00500,1,2021-09-01 00:00:00,0,105,52
227,13,1,3,674.80500,1,2021-09-01 01:00:00,0,227,52
349,13,1,3,681.85200,1,2021-09-01 02:00:00,0,349,52
471,13,1,3,671.65200,1,2021-09-01 03:00:00,0,471,52
593,13,1,3,694.65300,1,2021-09-01 04:00:00,0,593,52


In [7]:
client_raw = pd.read_csv(root+'client.csv')
client_dict = create_data_dictionary(client_raw)
client_dict

identify colames and dtypes: done
null count: done
data length and data example: done


,Feature,Data Type,null_count,null_count_perc,length_occurancy,data_example,unique_values
0,product_type,int64,0,0.00000,None,"[3, 1, 0, 2]",0.00010
1,county,int64,0,0.00000,None,"[0, 11, 7, 5]",0.00040
2,eic_count,int64,0,0.00000,None,"[14, 8, 12, 9]",0.01150
3,installed_capacity,float64,0,0.00000,None,"[41919.0, 1450.7714513705005, 2422.23312018854...",0.04660
4,is_business,int64,0,0.00000,None,"[1, 0]",0.00000
5,date,object,0,0.00000,"[(10, 41919)]","[2022-11-26, 2022-11-09, 2022-11-11, 2022-11-12]",0.01520
6,data_block_id,int64,0,0.00000,None,None,0.01520


In [8]:
e_prices_raw = pd.read_csv(root+'electricity_prices.csv')
e_prices_dict = create_data_dictionary(e_prices_raw)
e_prices_dict

identify colames and dtypes: done
null count: done
data length and data example: done


,Feature,Data Type,null_count,null_count_perc,length_occurancy_x,data_example_x,length_occurancy_y,data_example_y,unique_values
0,forecast_date,object,0,0.00000,"[(19, 15286)]","[2021-09-01 00:00:00, 2022-10-30 20:00:00, 202...","[(19, 15286)]","[2021-09-01 00:00:00, 2022-10-30 20:00:00, 202...",1.00000
1,euros_per_mwh,float64,0,0.00000,None,None,None,"[15286.0, 157.06417571634177, 121.148624971588...",0.72120
2,origin_date,object,0,0.00000,None,None,"[(19, 15286)]","[2021-08-31 00:00:00, 2022-10-29 20:00:00, 202...",1.00000
3,data_block_id,int64,0,0.00000,None,None,None,None,0.04170


In [9]:
forecast_weather_raw = pd.read_csv(root+'forecast_weather.csv')
forecast_weather_dict = create_data_dictionary(forecast_weather_raw)
forecast_weather_dict

identify colames and dtypes: done
null count: done
data length and data example: done


,Feature,Data Type,null_count,null_count_perc,length_occurancy_x,data_example_x,length_occurancy_y,data_example_y,unique_values
0,latitude,float64,0,0.00000,None,"[3424512.0, 58.64999999999994, 0.6873864546060...",None,"[3424512.0, 58.64999999999994, 0.6873864546060...",0.00000
1,longitude,float64,0,0.00000,None,"[3424512.0, 24.95000000000006, 2.0155647313596...",None,"[3424512.0, 24.95000000000006, 2.0155647313596...",0.00000
2,origin_datetime,object,0,0.00000,"[(25, 3424512)]","[2021-08-31 23:00:00+00:00, 2022-11-02 23:00:0...","[(25, 3424512)]","[2021-08-31 23:00:00+00:00, 2022-11-02 23:00:0...",0.00020
3,hours_ahead,int64,0,0.00000,None,None,None,"[1, 2, 27, 28]",0.00000
4,temperature,float64,0,0.00000,None,None,None,"[3424512.0, 5.743912920130524, 7.8442055645712...",0.11710
5,dewpoint,float64,0,0.00000,None,None,None,"[3424512.0, 2.4119463315858107, 7.121431885390...",0.12830
6,cloudcover_high,float64,0,0.00000,None,None,None,"[3424512.0, 0.39466538707738674, 0.44404250997...",0.06200
7,cloudcover_low,float64,0,0.00000,None,None,None,"[3424512.0, 0.43464533793686083, 0.43863461866...",0.11930
8,cloudcover_mid,float64,0,0.00000,None,None,None,"[3424512.0, 0.35906934760703213, 0.42015560381...",0.08990
9,cloudcover_total,float64,0,0.00000,None,None,None,"[3424512.0, 0.6819927145740771, 0.400962884601...",0.20390


In [10]:
gas_prices_raw = pd.read_csv(root+'gas_prices.csv')
gas_prices_dict = create_data_dictionary(gas_prices_raw)
gas_prices_dict

identify colames and dtypes: done
null count: done
data length and data example: done


,Feature,Data Type,null_count,null_count_perc,length_occurancy_x,data_example_x,length_occurancy_y,data_example_y,unique_values
0,forecast_date,object,0,0.00000,"[(10, 637)]","[2021-09-01, 2022-11-03, 2022-10-27, 2022-10-28]","[(10, 637)]","[2021-09-01, 2022-11-03, 2022-10-27, 2022-10-28]",1.00000
1,lowest_price_per_mwh,float64,0,0.00000,None,None,None,"[637.0, 95.03675039246468, 47.55229451330452, ...",0.64990
2,highest_price_per_mwh,float64,0,0.00000,None,None,None,"[637.0, 107.75463108320253, 54.743666079706344...",0.71590
3,origin_date,object,0,0.00000,None,None,"[(10, 637)]","[2021-08-31, 2022-11-02, 2022-10-26, 2022-10-27]",1.00000
4,data_block_id,int64,0,0.00000,None,None,None,None,1.00000


In [11]:
hist_weather_raw = pd.read_csv(root+'historical_weather.csv')
hist_weather_dict = create_data_dictionary(hist_weather_raw)
hist_weather_dict

identify colames and dtypes: done
null count: done
data length and data example: done


,Feature,Data Type,null_count,null_count_perc,length_occurancy,data_example,unique_values
0,datetime,object,0,0.00000,"[(19, 1710802)]","[2022-11-12 15:00:00, 2021-09-01 00:00:00, 202...",0.00890
1,temperature,float64,0,0.00000,None,None,0.00030
2,dewpoint,float64,0,0.00000,None,None,0.00030
3,rain,float64,0,0.00000,None,None,0.00000
4,snowfall,float64,0,0.00000,None,None,0.00000
5,surface_pressure,float64,0,0.00000,None,None,0.00060
6,cloudcover_total,int64,0,0.00000,None,None,0.00010
7,cloudcover_low,int64,0,0.00000,None,None,0.00010
8,cloudcover_mid,int64,0,0.00000,None,None,0.00010
9,cloudcover_high,int64,0,0.00000,None,None,0.00010


In [12]:
weather_station_to_county_mapping_raw = pd.read_csv(root+'weather_station_to_county_mapping.csv')
weather_station_to_county_mapping_raw_dict = create_data_dictionary(weather_station_to_county_mapping_raw)
weather_station_to_county_mapping_raw_dict

identify colames and dtypes: done
null count: done
data length and data example: done


,Feature,Data Type,null_count,null_count_perc,length_occurancy,data_example,unique_values
0,county_name,object,63,0.56250,"[(8.0, 29), (11.0, 7), (7.0, 6), (13.0, 4), (9...","[Harjumaa, Pärnumaa, Võrumaa, Saaremaa]",0.13390
1,longitude,float64,0,0.00000,None,None,0.12500
2,latitude,float64,0,0.00000,None,None,0.07140
3,county,float64,63,0.56250,None,None,0.13390
